# Character Animations
This is a bonus. Let's try to animate these characters.

In [1]:
import settings
from model import Story

# The last time json data was saved was step 4
story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

# Generate list of scene images

Let's stick the characters into a scene so that we can have lots of context.

In [2]:
from PIL import Image
import settings

def place_character_in_scene(image: Image, background: Image=None):
    width = settings.CHARACTER_ANIMATION_WIDTH
    height = settings.CHARACTER_ANIMATION_HEIGHT

    # Create a transparent background
    resize_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    if background:
        # Add background to the image
        scene = background.convert("RGBA")

        # Scale the scene to fit the normal dimensions while preserving aspect ratio
        scene_ratio = max(width / scene.width, height / scene.height)
        new_scene_size = (int(scene.width * scene_ratio), int(scene.height * scene_ratio))
        scene = scene.resize(new_scene_size)
        
        # Calculate the position to paste the scene image onto the background
        scene_offset = ((width - new_scene_size[0]) // 2, (height - new_scene_size[1]))
        
        # Paste the scene image onto the background
        resize_image.paste(scene, scene_offset, scene)


    # Calculate the position to paste the source image onto the background
    src_width, src_height = image.size
    offset = ((width - src_width) // 2, (height - src_height))
    # offset = ((width - src_width) // 2, (height - src_height) // 2)

    # Ensure the image has an alpha channel
    image = image.convert("RGBA")
    
    # Paste the source image onto the background
    resize_image.paste(image, offset, image)

    # Convert the image to RGB
    resize_image = resize_image.convert("RGB")

    return resize_image

In [ ]:
from IPython.display import display
from IPython.display import Image as IPImage
import rembg
from diffusers.utils import export_to_gif
import os
import random

for character in story.characters:
    act = random.choice(story.acts)
    scene = random.choice(act.scenes)

    src_image_path = f"{settings.STORY_DIR}/step_5/characters/{character.nickname}.png"
    src_image = Image.open(src_image_path)

    # Make the character a bit smaller
    src_image = src_image.resize((src_image.size[0]//3*2, src_image.size[1]//3*2))

    # Remove the background
    src_image = src_image.convert("RGBA")
    src_image = rembg.remove(src_image)

    background = Image.open(f"stories/my_story/step_6/scenes/{scene.scene_id}.png")

    scene_image = place_character_in_scene(src_image, background=background)
    # character_scene_images[character.name] = scene_image

    dst_image_path = f"{settings.STORY_DIR}/step_7/characters/{character.nickname}_scene.png"
    dir_name = os.path.dirname(dst_image_path)
    os.makedirs(dir_name, exist_ok=True)

    scene_image.save(dst_image_path)

    display(character.name)
    display(IPImage(dst_image_path))

# Generate list of animation descriptions
We will use vision to look at the character and scene. Then generate a description of the animation from that.

# Animate Each Character


In [ ]:
from model import Story
import settings

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

### Image/Text-to-video

#### CogVideoX-5b-I2V
https://huggingface.co/THUDM/CogVideoX-5b-I2V

#### i2vgen-xl
https://huggingface.co/docs/diffusers/main/en/using-diffusers/text-img2vid#i2vgen-xl


In [ ]:
from IPython.display import display, Markdown
from model_image2video import image_to_video
from utils import deindent
from PIL import Image



for c in story.characters:
    src_image_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}_scene.png"
    src_image = Image.open(src_image_path)
    
    prompt = f"epic full arc shot of the to the right of a {c.gender} {c.race} {c.role} age {c.age}. {c.physical_appearance}"
    # prompt = f"{c.animation_description}. {scene.background_animation}"
    # prompt = f"An slow epic arc shot to the right while zooming in. Grenades are exploding all around the ninja attacks and dust is flying everywhere. {scene.background_animation}"
    # prompt = f"a {c.gender} {c.race} speaks animatedly, gesturing with their hands. Lasers shoot out of their eyes and an explosion ensues."

    formatted_prompt = "\n".join([f"> {line}" for line in prompt.split("\n")])
    dst_image_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}.gif"
    dst_video_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}.mp4"
    guidance_scale = 7.0
    
    display(Markdown(deindent(f"""
        ---
                
        ## {c.name}

        **Role**: {c.role}
                            
        **Physical Apperaance**: {c.physical_appearance}

        **Description**: {c.description}

        **Personality**: {c.personality}

        ### Prompt:

        **guidance_scale**: {guidance_scale}

        {formatted_prompt}

    """)))
    
    image_to_video(prompt=prompt, image=src_image, gif_filename=dst_image_path, video_filename=dst_video_path, display_video=True, sequences=2)


---

## Meera 'Midnight' Singh

**Role**: Protagonist/Gamer

**Physical Apperaance**: Petite, with short, spiky black hair, expressive brown eyes, and a collection of gaming-themed tattoos on her arms. Often wears comfortable, neon-lit gaming attire.

**Description**: The protagonist, a teenage underdog gamer with extraordinary skills that connect her to an alternate reality.

**Personality**: Determined, resourceful, and initially introverted, with a growing sense of responsibility and leadership.

### Prompt:

**guidance_scale**: 7.0

> epic full arc shot of the to the right of a Female Indian Protagonist/Gamer age 17. Petite, with short, spiky black hair, expressive brown eyes, and a collection of gaming-themed tattoos on her arms. Often wears comfortable, neon-lit gaming attire.

  0%|          | 0/50 [00:00<?, ?it/s]